In [6]:
import pandas as pd
import yaml 

with open(yaml.full_load("../Paths.yaml")) as f:
    Paths = yaml.full_load(f)

baseDataPath = Paths["Paths"]["External"]

feature_ref_file = baseDataPath+"/E_MTAB_9964/feature_reference.TEW__47fcbc__Lipid_Hashing_Nuclei_MCF7_PC3_MDAMB231_DU145_cdna___TEW__3d9499__Lipid_Hashing_Nuclei_MCF7_PC3_MDAMB231_DU145_hto.csv"
library_file = baseDataPath+"/E_MTAB_9964/libraries.TEW__47fcbc__Lipid_Hashing_Nuclei_MCF7_PC3_MDAMB231_DU145_cdna___TEW__3d9499__Lipid_Hashing_Nuclei_MCF7_PC3_MDAMB231_DU145_hto.csv"

# 1 Alignment and genetic demultiplexing

![image info](../Pictures/GeneticDemultiplexing_pipeline.png)

## Alignment

In [ ]:
%%bash
cellranger count --id=SampleName \
   --fastqs=path/to/fastq/folder \
   --transcriptome=/opt/refdata-gex-GRCh38-2020-A

![image info](../Pictures/CountOutput.png)

## Genetic Demultiplexing

In [ ]:
%%bash
popscle dsc-pileup --sam possorted_genome_bam.bam --vcf ref_vcf.vcf --group-list filtered_features_bc_matrix/barcodes.tsv --out Pileup.plp
popscle freemuxlet --plp Pileup.plp --out identities.tsv --nsample 4

or

In [ ]:
%%bash
demuxlet --sam possorted_genome_bam.bam --vcf genotypes_vcf.vcf --field PL --out identities.tsv

**additional filters on VCF file may be useful**
- Allelic frequency 
- Quality


# 2 Alignment and tags assignment

![image info](../Pictures/TagsDemultiplexing_pipeline.png)

In [ ]:
%%bash
cellranger count --id=SampleName \
                   --libraries=library_file.csv \
                   --transcriptome=/opt/refdata-gex-GRCh38-2020-A \
                   --feature-ref=feature_ref_file.csv                   

In [7]:
pd.read_csv(library_file)

,fastqs,sample,library_type
0,/staging/leuven/stg_00002/lcb/ngs_runs/HiSeq40...,TEW__47fcbc__Lipid_Hashing_Nuclei_MCF7_PC3_MDA...,Gene Expression
1,/staging/leuven/stg_00002/lcb/ngs_runs/HiSeq40...,TEW__3d9499__Lipid_Hashing_Nuclei_MCF7_PC3_MDA...,Antibody Capture


In [8]:
pd.read_csv(feature_ref_file)

,id,name,read,pattern,sequence,feature_type
0,Hash-MULTI-1,Hash-MULTI-1,R2,^(BC),TTGTCACGGTAATTA,Antibody Capture
1,Hash-MULTI-2,Hash-MULTI-2,R2,^(BC),ATCGAACCGACAGAG,Antibody Capture
2,Hash-MULTI-3,Hash-MULTI-3,R2,^(BC),GGTCGAATATGTCGG,Antibody Capture
3,Hash-MULTI-4,Hash-MULTI-4,R2,^(BC),CTCAAGCATTATCAT,Antibody Capture


![image info](../Pictures/CountOutputTAGs.png)